### Dates and Times in Python
• The Python world has a number of available representations of dates, times, deltas,
and timespans.

• While the time series tools provided by Pandas tend to be the most useful for data science applications, it is helpful to see their relationship to other packages used in Python.

In [15]:
import numpy as np
import pandas as pd

### Native Python dates and times: datetime and dateutil
• Python’s basic objects for working with dates and times reside in the built-in date time module.

• Along with the third-party dateutil module, you can use it to quickly perform a host of useful functionalities on dates and times.

• For example, you can manually build a date using the datetime type:

In [16]:
from datetime import datetime
datetime(year = 2018, month = 5, day = 17)

datetime.datetime(2018, 5, 17, 0, 0)

In [17]:
from dateutil import parser
date = parser.parse("4th of May, 1994")
date

datetime.datetime(1994, 5, 4, 0, 0)

• Once you have a datetime object, you can do things like printing the day of the week:

In [18]:
date.strftime('%A')

'Wednesday'

### Typed arrays of times: NumPy’s datetime64 

• The weaknesses of Python’s datetime format inspired the NumPy team to add a set of native time series data type to NumPy.

• The datetime64 dtype encodes dates as 64-bit integers, and thus allows arrays of dates to be represented very compactly.

• The datetime64 requires a very specific input format:

In [19]:
date = np.array("2017-04-04", dtype = np.datetime64)
date

array('2017-04-04', dtype='datetime64[D]')

• Once we have this date formatted, however, we can quickly do vectorized operations on it:

In [20]:
date + np.arange(10)

array(['2017-04-04', '2017-04-05', '2017-04-06', '2017-04-07',
       '2017-04-08', '2017-04-09', '2017-04-10', '2017-04-11',
       '2017-04-12', '2017-04-13'], dtype='datetime64[D]')

In [21]:
# day-based datetime:
np.datetime64('2020-01-19')

numpy.datetime64('2020-01-19')

In [22]:
# minute-based datetime:
np.datetime64('2021-01-20 19:22')

numpy.datetime64('2021-01-20T19:22')

### Dates and times in Pandas: Best of both worlds
• Pandas builds upon all the tools just discussed to provide a Timestamp object, which combines the ease of use of datetime and dateutil with the efficient storage and vectorized interface of numpy.datetime64.

• From a group of these Timestamp objects, Pandas can construct a DatetimeIndex that can be used to index data in a Series or DataFrame.

In [23]:
date = pd.to_datetime("7th of July, 2019")
date

Timestamp('2019-07-07 00:00:00')

In [24]:
date.strftime('%A')

'Sunday'

• Additionally, we can do NumPy-style vectorized operations directly on this same object:

In [25]:
date + pd.to_timedelta(np.arange(12), 'M')

DatetimeIndex(['2019-07-07 00:00:00', '2019-08-06 10:29:06',
               '2019-09-05 20:58:12', '2019-10-06 07:27:18',
               '2019-11-05 17:56:24', '2019-12-06 04:25:30',
               '2020-01-05 14:54:36', '2020-02-05 01:23:42',
               '2020-03-06 11:52:48', '2020-04-05 22:21:54',
               '2020-05-06 08:51:00', '2020-06-05 19:20:06'],
              dtype='datetime64[ns]', freq=None)

### Pandas Time Series: Indexing by Time

Where the Pandas time series tools really become useful is when you begin to index data by timestamps.

In [26]:
index = pd.DatetimeIndex(['2014-07-04', '2014-08-04', '2015-07-04', '2015-08-04'])
data = pd.Series([0, 1, 2, 3], index = index)
data

2014-07-04    0
2014-08-04    1
2015-07-04    2
2015-08-04    3
dtype: int64

• Now that we have this data in a Series , we can make use of any of the Series indexing patterns we discussed in previous sections, passing values that can be coerced into dates:

In [27]:
data['2014-07-04':'2015-07-04']

2014-07-04    0
2014-08-04    1
2015-07-04    2
dtype: int64

• There are additional special date-only indexing operations, such as passing a year to obtain a slice of all data from that year:

In [28]:
data['2015']

2015-07-04    2
2015-08-04    3
dtype: int64

### Pandas Time Series Data Structures

• For time stamps, Pandas provides the Timestamp type. As mentioned before, it is essentially a replacement for Python’s native datetime , but is based on the more efficient numpy.datetime64 data type. The associated index structure is DatetimeIndex.

• For time periods, Pandas provides the Period type. This encodes a fixed-frequency interval based on numpy.datetime64 . The associated index structure is PeriodIndex .

• For time deltas or durations, Pandas provides the Timedelta type. Timedelta is a more efficient replacement for Python’s native datetime.timedelta type, and is based on numpy.timedelta64. The associated index structure is TimedeltaIndex.

• The most fundamental of these date/time objects are the Timestamp and DatetimeIndex objects.

• While these class objects can be invoked directly, it is more common to use the pd.to_datetime() function, which can parse a wide variety of formats.

• Passing a single date to pd.to_datetime() yields a Timestamp ; passing a series of dates by default yields a DatetimeIndex:

In [31]:
dates = pd.to_datetime(['2017, 5, 10', '5th of Aug, 2020', '2015-Jan-19', '02-03-2019', '20190303'])
dates

DatetimeIndex(['2017-05-10', '2020-08-05', '2015-01-19', '2019-02-03',
               '2019-03-03'],
              dtype='datetime64[ns]', freq=None)

• Any DatetimeIndex can be converted to a PeriodIndex with the to_period() function with the addition of a frequency code; here we’ll use 'D' to indicate daily frequency.

In [32]:
dates.to_period('M')

PeriodIndex(['2017-05', '2020-08', '2015-01', '2019-02', '2019-03'], dtype='period[M]', freq='M')

• A TimedeltaIndex is created, for example, when one date is subtracted from another:

In [33]:
dates - dates[0]

TimedeltaIndex(['0 days', '1183 days', '-842 days', '634 days', '662 days'], dtype='timedelta64[ns]', freq=None)

### Regular sequences: pd.date_range()

• To make the creation of regular date sequences more convenient, Pandas offers a few functions for this purpose: pd.date_range() for timestamps, pd.period_range() for periods, and pd.timedelta_range() for time deltas.

• We’ve seen that Python’s range() and NumPy’s np.arange() turn a startpoint, endpoint, and optional stepsize into a sequence.

• Similarly, pd.date_range() accepts a start date, an end date, and an optional frequency code to create a regular sequence of dates.

• By default, the frequency is one day:

In [36]:
pd.date_range('01-01-2100', '20-01-2100')

DatetimeIndex(['2100-01-01', '2100-01-02', '2100-01-03', '2100-01-04',
               '2100-01-05', '2100-01-06', '2100-01-07', '2100-01-08',
               '2100-01-09', '2100-01-10', '2100-01-11', '2100-01-12',
               '2100-01-13', '2100-01-14', '2100-01-15', '2100-01-16',
               '2100-01-17', '2100-01-18', '2100-01-19', '2100-01-20'],
              dtype='datetime64[ns]', freq='D')

• Alternatively, the date range can be specified not with a start- and endpoint, but with a startpoint and a number of periods:

In [39]:
pd.date_range('01-01-2019', periods = 10)

DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04',
               '2019-01-05', '2019-01-06', '2019-01-07', '2019-01-08',
               '2019-01-09', '2019-01-10'],
              dtype='datetime64[ns]', freq='D')

• You can modify the spacing by altering the freq argument, which defaults to D.

• For example, here we will construct a range of hourly timestamps

In [40]:
pd.date_range('20-03-2018', periods = 10, freq = 'Y')

DatetimeIndex(['2018-12-31', '2019-12-31', '2020-12-31', '2021-12-31',
               '2022-12-31', '2023-12-31', '2024-12-31', '2025-12-31',
               '2026-12-31', '2027-12-31'],
              dtype='datetime64[ns]', freq='A-DEC')